In [6]:
import requests 
import csv 
#from bs4 import BeautifulSoup
import os 
import pandas as pd
import numpy as np
import re 


In [7]:
# BASKETBALL REFERENCES DATA
#NBATeamNamesAB = ["TOR"]
NBATeamNamesAB = ["ATL", "BRK", "BOS", "CHO", "CHI", "CLE", "DAL", "DEN", "DET", "GSW", "HOU", "IND", "LAC", "LAL", "MEM", "MIA", "MIL", "MIN", "NOP", "NYK", "OKC", "ORL", "PHI", "PHO", "POR", "SAC", "SAS", "TOR", "UTA", "WAS"]


df = pd.DataFrame()

# FINAL DATAFRAME TO RETURN
Final_DF = pd.DataFrame()


# BASKETBALL REFERENCES SCRAPE
for team in NBATeamNamesAB:
    urlRoster = f"https://www.basketball-reference.com/teams/{team}/2019.html"
    RosterTable = pd.read_html(urlRoster)
    RosterDataFrame = RosterTable[0]
    RosterDataFrame["Team"] = team
    df = pd.concat([df , RosterDataFrame])

df.reset_index(inplace = True)
df = df.rename(index=str, columns={"Player": "Name"})


In [8]:
df.head()

index  No.             Name Pos   Ht   Wt         Birth Date Unnamed: 6  \
0      0   10     Jaylen Adams  PG  6-2  190        May 4, 1996         us   
1      1    1  Justin Anderson  SF  6-6  230  November 19, 1993         us   
2      2   24    Kent Bazemore  SG  6-5  201       July 1, 1989         us   
3      3   95  DeAndre' Bembry  SG  6-6  210       July 4, 1994         us   
4      4   15     Vince Carter  SF  6-6  220   January 26, 1977         us   

  Exp                       College Team  
0   R    St. Bonaventure University  ATL  
1   3        University of Virginia  ATL  
2   6       Old Dominion University  ATL  
3   2     Saint Joseph's University  ATL  
4  20  University of North Carolina  ATL

In [9]:
def dataset_func (team, year, SeasonType):
    urlPlStats = f"https://www.espn.com/nba/team/stats/_/name/{team}/season/{year}/seasontype/{SeasonType}"
    dirty_data = pd.read_html(urlPlStats)
    #cleaning(Final_DF, dirty_data)

    # Player's name and number
    dfSnPt1 = dirty_data[1]
    # Player's information
    dfSnPt2 = dirty_data[3]
    # Merging the two together
    dfSnmerge = pd.merge(dfSnPt1, dfSnPt2, left_index=True, right_index=True)

    #Renaming columns 
    dfSnmerge = dfSnmerge.rename(index=str, columns={"GP":"Games Played", "GS":"Games Started", "MIN": "Minutes Per Game",
                                                      "PTS": "Points_Per_Game", "OR": "Offensive Rebounds", 
                                                     "DR": "Defensive Rebounds", "REB": "Rebounds Per Game", 
                                                     "AST":"Assists Per Game", "STL": "Steal Per Game",
                                                     "BLK": "Blocks Per Game", "TO": "Turnovers Per Games",
                                                     "PF": "Fouls Per Game", "AST/TO Season": "Assists to Turnover Ratio",
                                                     "PER": "Player Efficiency Rating" })
    dfSnmergeName = dfSnmerge["Name"].str.split(" ", n = 2, expand = True)
    dfSnNewMerge = pd.merge(dfSnmerge, dfSnmergeName, left_index=True, right_index=True)
    dfSnNewMerge = dfSnNewMerge.drop(['Name'], axis=1)

    #Renaming columns first name, last name, position
    dfSnNewMerge = dfSnNewMerge.rename(columns= {dfSnNewMerge.columns[14]: "First", dfSnNewMerge.columns[15]: "Last", dfSnNewMerge.columns[16]: "Position"})
    dfSnNewMerge['Name'] = dfSnNewMerge['First'].str.cat(dfSnNewMerge['Last'],sep=" ")
    dfSnNewMerge = dfSnNewMerge.drop(['Last'], axis=1)
    dfSnNewMerge = dfSnNewMerge.drop(['First'], axis=1)

    if SeasonType == "2":
        Stype = "Season"
    if SeasonType == "3":
        Stype = "Playoffs"
   
    dfSnNewMerge.columns = [str(col) + " " + Stype for col in dfSnNewMerge.columns if col != "Name"] + ["Name"]
    return dfSnNewMerge


In [11]:
espnTeamNames = [ "TOR", "BKN", "NY", "PHI", "CHI", "CLE", "DET" , "IND", "MIL", "ATL", "CHA",
                 "MIA", "ORL", "WSH", "GS", "LAC", "LAL", "PHX", "SAC" , "HOU", "DAL", "MEM", 
                 "NO", "SA", "DEN", "MIN", "OKC", "POR", "UTAH"]
years =["2019", "2018", "2017", "2016", "2015", "2014", "2013", "2012", "2011", "2010",
        "2009", "2008", "2007", "2006", "2005"]

master_df = pd.DataFrame()
 
for team in espnTeamNames:
    for year in years:
        #print(year , team)
        df_team = dataset_func(team, year, "2").merge(dataset_func (team, year, "3"), how = "left" , on = "Name")
        df_team["Year"] = year
        df_team = df_team[:-1] #drop total row

        master_df = pd.concat([master_df , df_team] , ignore_index= True)


HTTPError: HTTP Error 504: Gateway Time-out

In [ ]:
master_df.to_csv("df_years_teams.csv")

In [7]:
final_df = df.merge(master_df , on = "Name")

In [8]:
pd.set_option('display.max_rows', 500)
final_df.head(40)

index  No.             Name Pos    Ht   Wt          Birth Date Unnamed: 6  \
0       0   10     Jaylen Adams  PG   6-2  190         May 4, 1996         us   
1       1    1  Justin Anderson  SF   6-6  230   November 19, 1993         us   
2       1    1  Justin Anderson  SF   6-6  230   November 19, 1993         us   
3       1    1  Justin Anderson  SF   6-6  230   November 19, 1993         us   
4       1    1  Justin Anderson  SF   6-6  230   November 19, 1993         us   
5       1    1  Justin Anderson  SF   6-6  230   November 19, 1993         us   
6       2   24    Kent Bazemore  SG   6-5  201        July 1, 1989         us   
7       2   24    Kent Bazemore  SG   6-5  201        July 1, 1989         us   
8       2   24    Kent Bazemore  SG   6-5  201        July 1, 1989         us   
9       2   24    Kent Bazemore  SG   6-5  201        July 1, 1989         us   
10      2   24    Kent Bazemore  SG   6-5  201        July 1, 1989         us   
11      2   24    Kent Bazemore  SG   6-5  201        July 1, 1989         us   
12      2   24    Kent Bazemore  SG   6-5  201        July 1, 1989         us   
13      2   24    Kent Bazemore  SG   6-5  201        July 1, 1989         us   
14      3   95  DeAndre' Bembry  SG   6-6  210        July 4, 1994         us   
15      3   95  DeAndre' Bembry  SG   6-6  210        July 4, 1994         us   
16      3   95  DeAndre' Bembry  SG   6-6  210        July 4, 1994         us   
17      4   15     Vince Carter  SF   6-6  220    January 26, 1977         us   
18      4   15     Vince Carter  SF   6-6  220    January 26, 1977         us   
19      4   15     Vince Carter  SF   6-6  220    January 26, 1977         us   
20      4   15     Vince Carter  SF   6-6  220    January 26, 1977         us   
21      4   15     Vince Carter  SF   6-6  220    January 26, 1977         us   
22      4   15     Vince Carter  SF   6-6  220    January 26, 1977         us   
23      4   15     Vince Carter  SF   6-6  220    January 26, 1977         us   
24      4   15     Vince Carter  SF   6-6  220    January 26, 1977         us   
25      4   15     Vince Carter  SF   6-6  220    January 26, 1977         us   
26      4   15     Vince Carter  SF   6-6  220    January 26, 1977         us   
27      4   15     Vince Carter  SF   6-6  220    January 26, 1977         us   
28      4   15     Vince Carter  SF   6-6  220    January 26, 1977         us   
29      4   15     Vince Carter  SF   6-6  220    January 26, 1977         us   
30      4   15     Vince Carter  SF   6-6  220    January 26, 1977         us   
31      4   15     Vince Carter  SF   6-6  220    January 26, 1977         us   
32      4   15     Vince Carter  SF   6-6  220    January 26, 1977         us   
33      4   15     Vince Carter  SF   6-6  220    January 26, 1977         us   
34      5   20     John Collins  PF  6-10  235  September 23, 1997         us   
35      5   20     John Collins  PF  6-10  235  September 23, 1997         us   
36      6    4    Deyonta Davis   C  6-11  237    December 2, 1996         us   
37      6    4    Deyonta Davis   C  6-11  237    December 2, 1996         us   
38      6    4    Deyonta Davis   C  6-11  237    December 2, 1996         us   
39      7   14   Dewayne Dedmon   C   7-0  245     August 12, 1989         us   

   Exp                            College  ... Rebounds Per Game Playoffs  \
0    R         St. Bonaventure University  ...                        1.8   
1    3             University of Virginia  ...                        1.3   
2    3             University of Virginia  ...                        4.0   
3    3             University of Virginia  ...                        1.8   
4    3             University of Virginia  ...                        2.9   
5    3             University of Virginia  ...                        4.0   
6    6            Old Dominion University  ...                        3.9   
7    6            Old Dominion University  ...                    

In [9]:
# 

In [10]:
final_df.to_csv("final_df.csv")

In [ ]:
# dfSnNewMerge = dfSnNewMerge.drop(['Last'], axis=1)
# dfSnNewMerge = dfSnNewMerge.rename(columns= {dfSnNewMerge.columns[14]: "First", dfSnNewMerge.columns[15]: "Last", dfSnNewMerge.columns[16]: "Position"})
#dfSnmerge = dfSnmerge.rename(index=str, columns={"GP":"Games Played", "GS":"Games Started", "MIN": "Minutes Per Game",
#                                                       "PTS": "Points Per Game", "OR": "Offensive Rebounds", 
#                                                      "DR": "Defensive Rebounds", "REB": "Rebounds Per Game", 
#                                                      "AST":"Assists Per Game", "STL": "Steal Per Game",
#                                                      "BLK": "Blocks Per Game", "TO": "Turnovers Per Games",
#                                                      "PF": "Fouls Per Game", "AST/TO Season": "Assists to Turnover Ratio",
#                                                      "PER": "Player Efficiency Rating" })

In [2]:
final_df_cleaning = final_df.rename(index=str, columns={"POS": "Position","Points Per Game Season" : "Points_season","Player Efficiency Rating Season":"PER", "Ht": "Height", "Wt": "Weight", "Unnamed: 6" : "Country Abbreviation", "Position Season": "Position", "No.": "Player Jersey Number", "Exp": "Years of Experience"})
final_df_cleaning = final_df_cleaning.drop(['index'], axis=1)
final_df_cleaning = final_df_cleaning.drop(['Position Playoffs'], axis=1)
final_df_cleaning.head()

NameError: name 'final_df' is not defined

In [1]:
final_df_cleaning.to_csv("df_final_df_cleaning_PROJECT.csv")

NameError: name 'final_df_cleaning' is not defined

In [11]:
len(final_df["Name"].unique())

478

In [12]:
len(df["Name"].unique())

530

In [13]:
# def cleaning(Final_DF, dirty_data):
    
#     # Basketball references roster
#     #NBATeamNamesAB = ["ATL", "BRK", "BOS", "CHO", "CHI", "CLE", "DAL", "DEN", "DET", "GSW", "HOU", "IND", "LAC", "LAL", "MEM", "MIA", "MIL", "MIN", "NOP", "NYK", "OKC", "ORL", "PHI", "PHO", "POR", "SAC", "SAS", "TOR", "UTA", "WAS"]
#     NBATeamNamesAB = ["TOR"]


#      
#         RosterDataFrame["Team"] = team
#         df = pd.concat([df , RosterDataFrame])
    
#     df.reset_index(inplace = True)
#     df = df.rename(index=str, columns={"Player": "Name"})
    
    
    
#     # Player's name and number
#     dfSnPt1 = dirty_data[1]
#     # Player's information
#     dfSnPt2 = dirty_data[3]
#     # Merging the two together
#     dfSnmerge = pd.merge(dfSnPt1, dfSnPt2, left_index=True, right_index=True)
    
#     #Renaming columns 
#     dfSnmerge = dfSnmerge.rename(index=str, columns={"GP":"Games Played Season", "GS":"Games Started Season", "MIN": "Minutes Per Game Season", "PTS": "Points Per Game Season", "OR": "Offensive Rebounds Season", "DR": "Defensive Rebounds Season", "REB": "Rebounds Per Game Season", "AST":"Assists Per Game Season", "STL": "Steal Per Game Season", "BLK": "Blocks Per Game Season", "TO": "Turnovers Per Games Season", "PF": "Fouls Per Game Season", "AST/TO Season": "Assists to Turnover Ratio Season", "PER": "Player Efficiency Rating Season" })
#     dfSnmergeName = dfSnmerge["Name"].str.split(" ", n = 2, expand = True)
#     dfSnNewMerge = pd.merge(dfSnmerge, dfSnmergeName, left_index=True, right_index=True)
#     dfSnNewMerge = dfSnNewMerge.drop(['Name'], axis=1)
    
#     #Renaming columns first name, last name, position
#     dfSnNewMerge = dfSnNewMerge.rename(columns= {dfSnNewMerge.columns[14]: "First", dfSnNewMerge.columns[15]: "Last", dfSnNewMerge.columns[16]: "Position"})
#     dfSnNewMerge['Name'] = dfSnNewMerge['First'].str.cat(dfSnNewMerge['Last'],sep=" ")
#     dfSnNewMerge = dfSnNewMerge.drop(['Last'], axis=1)
#     dfSnNewMerge = dfSnNewMerge.drop(['First'], axis=1)
#     dfFnSnMerge = pd.merge(df, dfSnNewMerge, on = "Name")
#     joined_data = dfSnNewMerge.merge(df, 'outer', 'Name')
#     Finaldf = pd.concat(joined_data, axis=1)

In [14]:
#BothStats.dropna(how='any').shape

In [14]:
#ESPN's Basketball Power Index 

#Tas

urlScores = "https://www.espn.com/nba/story/_/page/Basketball-Power-Index/espn-nba-basketball-power-index"
Scores = pd.read_html(urlScores)
Scores = Scores[0]
Scores = Scores.drop(['Unnamed: 1', "2-WKCHNG"], axis=1)
#Scores.head()
#Scores = Scores['Ranke'].str.split(' ', 1, expand=True)
Rank_split_Scores = Scores["RANK"].str.split(" ", n = 1, expand = True)
Merged_Scores = pd.merge(Rank_split_Scores, Scores, left_index=True, right_index=True)
Merged_Scores.head()
Merged_Scores = Merged_Scores.rename(columns= {Merged_Scores.columns[0]: "Number", Merged_Scores.columns[1]: "TeamAB"})
Merged_Scores = Merged_Scores.drop(['RANK', 'Number'], axis=1)


Merged_Scores.head(30)






TeamAB  OFF  DEF  BPI
0     MIL  3.3  4.4  7.7
1      GS  5.1  1.0  6.1
2     TOR  2.5  3.5  6.0
3     HOU  4.8  0.8  5.6
4     UTH  0.2  4.2  4.5
5     BOS  1.2  3.0  4.2
6     POR  3.6  0.2  3.8
7     DEN  2.9  0.9  3.8
8     OKC  0.1  2.9  3.1
9     PHI  2.0  1.0  3.0
10    IND -0.8  3.3  2.5
11     SA  2.0 -0.4  1.7
12    LAC  1.9 -1.2  0.7
13    MIA -2.8  2.5 -0.3
14     NO  1.1 -1.7 -0.6
15    ORL -2.1  1.4 -0.6
16    LAL -2.1  1.4 -0.7
17    MIN  1.5 -2.4 -0.9
18    DET -1.6  0.5 -1.1
19    BKN -1.0 -0.2 -1.2
20    DAL -0.9 -0.3 -1.2
21    SAC -0.4 -1.0 -1.4
22    CHA  0.7 -2.2 -1.5
23    WSH  0.9 -3.3 -2.4
24    MEM -3.9  1.5 -2.4
25    ATL -2.3 -3.7 -6.0
26    CHI -5.1 -2.5 -7.6
27    PHX -3.8 -4.2 -7.9
28     NY -5.2 -3.0 -8.2
29    CLE -2.0 -6.6 -8.6

In [15]:

#Tasks: 


#Fox News Sports Team Stats Pull
FoxTeamStats = "https://www.foxsports.com/nba/team-stats?season=2018&category=SCORING&group=1&time=0"
TeamStats = pd.read_html(FoxTeamStats)
TeamStats = TeamStats[0]


#Split Team Info and Index in SCORING cell
Rank_Team_Stats = TeamStats["SCORING"].str.split(" ", n = 1, expand = True)

#Rename Column to MixName to Split Again 
Rank_Team_Stats = Rank_Team_Stats.rename(columns= {Rank_Team_Stats.columns[1]: "TeamMix"})

#ISSUE: Split again doesn't work because some team names. Need Regex.  


#Rank_Team_Stats_Split = Rank_Team_Stats["TeamMix"].str.split(" ", n = 2, expand = True)
#Rank_Team_Stats_Split.head()
#Cleaned_FoxTeam_Stats = pd.merge(Rank_Team_Stats_Split, Rank_Team_Stats, left_index=True, right_index=True)
#Cleaned_FoxTeam_Stats.head()
#Rank_Team_Stats.head()
TeamStats.head(30)

SCORING  GP    PPG  PPG ALLOW  PPG DIFF  PTS OFF TO  \
0       1  Milwaukee  MIL  82  118.1      109.3       8.8        1366   
1    2  Golden State  GSW  82  117.7      111.2       6.5        1335   
2     3  New Orleans  NOP  82  115.4      116.8      -1.4        1322   
3    4  Philadelphia  PHI  82  115.2      112.5       2.7        1257   
4     5  Los Angeles  LAC  82  115.1      114.3       0.8        1271   
5        6  Portland  POR  82  114.7      110.5       4.2        1232   
6   7  Oklahoma City  OKC  82  114.5      111.1       3.4        1578   
7         8  Toronto  TOR  82  114.4      108.4       6.0        1414   
8      9  Sacramento  SAC  82  114.2      115.3      -1.1        1581   
9     10  Washington  WAS  82  114.0      116.9      -2.9        1454   
10       11  Houston  HOU  82  113.9      109.1       4.8        1480   
11       12  Atlanta  ATL  82  113.3      119.4      -6.1        1399   
12     13  Minnesota  MIN  82  112.5      114.0      -1.5        1430   
13        14  Boston  BOS  82  112.4      108.0       4.4        1486   
14      15  Brooklyn  BKN  82  112.2      112.3      -0.1        1263   
15   16  Los Angeles  LAL  82  111.8      113.5      -1.7        1379   
16   17  San Antonio  SAS  82  111.7      110.0       1.7        1157   
17          17  Utah  UTA  82  111.7      106.5       5.2        1347   
18     19  Charlotte  CHA  82  110.7      111.8      -1.1        1321   
19        19  Denver  DEN  82  110.7      106.7       4.0        1290   
20        21  Dallas  DAL  82  108.9      110.1      -1.2        1195   
21       22  Indiana  IND  82  108.0      104.7       3.3        1558   
22       23  Phoenix  PHX  82  107.5      116.8      -9.3        1492   
23       24  Orlando  ORL  82  107.3      106.6       0.7        1250   
24       25  Detroit  DET  82  107.0      107.3      -0.3        1296   
25         26  Miami  MIA  82  105.7      105.9      -0.2        1272   
26       27  Chicago  CHI  82  104.9      113.4      -8.5        1243   
27      28  New York  NYK  82  104.6      113.8      -9.2        1208   
28     29  Cleveland  CLE  82  104.5      114.1      -9.6        1183   
29       30  Memphis  MEM  82  103.5      106.1      -2.6        1405   

    PTS IN PAINT  SEC CHANCE PTS    FG%  FG% ALLOW   3FG%  3FG% ALLOW    FT%  \
0           4374             960  0.476      0.433  0.353       0.361  0.773   
1           3546            1031  0.491      0.444  0.385       0.347  0.801   
2           4774            1175  0.473      0.466  0.344       0.364  0.761   
3           4050            1159  0.471      0.455  0.359       0.342  0.771   
4           4202            1079  0.471      0.457  0.388       0.343  0.792   
5           4046            1230  0.467      0.457  0.359       0.359  0.814   
6           4250            1271  0.454      0.460  0.348       0.356  0.713   
7           3980            1037  0.474      0.449  0.366       0.345  0.804   
8           4252            1124  0.464      0.466  0.378       0.348  0.726   
9           4106            1017  0.468      0.480  0.341       0.370  0.768   
10          3454            1074  0.449      0.466  0.356       0.340  0.791   
11          4198            1155  0.451      0.473  0.352       0.359  0.752   
12          4062            1235  0.456      0.467  0.351       0.378  0.787   
13          3676            1022  0.465      0.448  0.365       0.344  0.802   
14          4002            1135  0.449      0.456  0.353       0.341  0.745   
15          4562            1051  0.470      0.452  0.333       0.349  0.699   
16          3548             948  0.478      0.463  0.392       0.359  0.819   
17          4084            1089  0.468      0.452  0.356       0.355  0.736   
18          3836            1065  0.448      0.470  0.351       0.361  0.797   
19          4280            1274  0.466      0.458  0.351       0.339  0.755   
20          3654            1061  0.447      0.459  0.340       0.344  0.7

In [18]:
#Basketball Reference Box Scores 

#Tasks: 
# - Take out '*' through Regex 



FoxTeamStats = "https://www.basketball-reference.com/boxscores/"
TeamStats = pd.read_html(FoxTeamStats)
EastConTeamStats = TeamStats[5]
EastConTeamStats = EastConTeamStats.rename(index=str, columns={"Eastern Conference":"Team"})
#EastConTeamStats

WestConTeamStats = TeamStats[6]
WestConTeamStats = WestConTeamStats.rename(index=str, columns={"Western Conference":"Team"})
#WestConTeamStats
EastWest = [WestConTeamStats, EastConTeamStats]
LeagueStats = pd.concat(EastWest)
LeagueStats.head(40)


Team   W   L   W/L%    GB   PS/G   PA/G
0    Golden State Warriors*  57  25  0.695     —  117.7  111.2
1           Denver Nuggets*  54  28  0.659   3.0  110.7  106.7
2          Houston Rockets*  53  29  0.646   4.0  113.9  109.1
3   Portland Trail Blazers*  53  29  0.646   4.0  114.7  110.5
4                Utah Jazz*  50  32  0.610   7.0  111.7  106.5
5    Oklahoma City Thunder*  49  33  0.598   8.0  114.5  111.1
6     Los Angeles Clippers*  48  34  0.585   9.0  115.1  114.3
7        San Antonio Spurs*  48  34  0.585   9.0  111.7  110.0
8          Sacramento Kings  39  43  0.476  18.0  114.2  115.3
9        Los Angeles Lakers  37  45  0.451  20.0  111.8  113.5
10   Minnesota Timberwolves  36  46  0.439  21.0  112.5  114.0
11         Dallas Mavericks  33  49  0.402  24.0  108.9  110.1
12        Memphis Grizzlies  33  49  0.402  24.0  103.5  106.1
13     New Orleans Pelicans  33  49  0.402  24.0  115.4  116.8
14             Phoenix Suns  19  63  0.232  38.0  107.5  116.8
0          Milwaukee Bucks*  60  22  0.732     —  118.1  109.3
1          Toronto Raptors*  58  24  0.707   2.0  114.4  108.4
2       Philadelphia 76ers*  51  31  0.622   9.0  115.2  112.5
3           Boston Celtics*  49  33  0.598  11.0  112.4  108.0
4           Indiana Pacers*  48  34  0.585  12.0  108.0  104.7
5            Orlando Magic*  42  40  0.512  18.0  107.3  106.6
6            Brooklyn Nets*  42  40  0.512  18.0  112.2  112.3
7          Detroit Pistons*  41  41  0.500  19.0  107.0  107.3
8         Charlotte Hornets  39  43  0.476  21.0  110.7  111.8
9                Miami Heat  39  43  0.476  21.0  105.7  105.9
10       Washington Wizards  32  50  0.390  28.0  114.0  116.9
11            Atlanta Hawks  29  53  0.354  31.0  113.3  119.4
12            Chicago Bulls  22  60  0.268  38.0  104.9  113.4
13      Cleveland Cavaliers  19  63  0.232  41.0  104.5  114.1
14          New York Knicks  17  65  0.207  43.0  104.6  113.8

In [18]:
#Task of merging all Team data into a Dataframe 
# Might need to create a dictionary to match up team abbreviations with full team name 